In [ ]:
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np
import glob
from scipy import interpolate
from scipy.optimize import curve_fit
from functions import Andor_calib, Adam_calib

In [ ]:
dic = {}
fps = glob.glob('Calibration.mat')
for idx, fp in enumerate(fps):
    with h5.File(fp, 'r') as f:
        for i in f.keys():
            dic[i] = np.array(f[i])
        #stages = np.array(f['Settings']['Axes']['Names'])
            #positions = np.array(f['Settings']['Axes']['Positions'])
            #data_t = np.array(f['Data'][:,:,:])
            #data_t = data_t[:,:,:]
            #if idx == 0:
                #data= np.zeros((len(fps),*data_t.shape))


In [ ]:
for i in dic.keys():
    print(i, dic[i].shape)

In [ ]:
cmap = plt.get_cmap('nipy_spectral')
cs = cmap(np.linspace(0,1,8))
cs = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'black']
fig, ax = plt.subplots(4, figsize=(10,10))
for j in range(8):
    ax[0].plot(dic['Enq'][j,:], label=dic['Steps'][:,j], color = cs[j])
    ax[1].plot(dic['I'][j,:], label=dic['Steps'][:,j], color = cs[j])
    ax[2].plot(dic['lq'][j,:], label=dic['Steps'][:,j], color = cs[j])
    ax[3].plot(dic['wq'][j,:], label=dic['Steps'][:,j], color = cs[j])

ax[0].set_ylabel('Enq')
ax[1].set_ylabel('I')
ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax[2].set_ylabel('lq')
ax[3].set_ylabel('wq')


In [ ]:
calib = []
for steps in range(8):
    calib.append(interpolate.interp1d(np.arange(2048),dic['lq'][j,:]))

In [ ]:
calib[0](np.arange(0,2048,100))

In [ ]:
def pol3(x, a, b, c, d):
    return a*x**3 + b*x**2 + c*x + d

In [ ]:
pcov, ppat = curve_fit(pol3, np.arange(0,2048,200), calib[0](np.arange(0,2048,200)))

In [ ]:
plt.plot(np.linspace(0,2000), calib[0](np.linspace(0,2000)))
plt.plot(np.arange(0,2048,200), pol3(np.arange(0,2048,200), *pcov), 'rx')


In [ ]:
calib_coeff = np.ones((8,4))
for s in range(8):
    pcov, ppat = curve_fit(pol3, np.arange(2048),dic['lq'][s,:])
    calib_coeff[s,:] = pcov
    plt.plot(np.linspace(0,2000), calib[0](np.linspace(0,2000)), color=cs[s])
    plt.plot(np.arange(0,2048,200), pol3(np.arange(0,2048,200), *pcov), marker = 'x', color=cs[s])
    plt.plot(np.arange(2048),dic['lq'][s,:], color=cs[s])
    

In [ ]:
fig, ax = plt.subplots(4, figsize=(10,10))
for coef in range(4):
    ax[coef].plot(dic['Steps'].T,calib_coeff[:,coef])

In [ ]:
def line(x, m,c):
    return m*x + c

In [ ]:
fig, ax = plt.subplots(4, figsize=(10,10))
offset = np.zeros((4,2))
for coef in range(4):
    ax[coef].plot(dic['Steps'].T,calib_coeff[:,coef])
    pcov, ppot = curve_fit(line, np.array(dic['Steps'])[0].T,np.array(calib_coeff[:,coef]))
    ax[coef].plot(dic['Steps'].T, line(dic['Steps'].T, *pcov),'x')
    offset[coef,:] = pcov


In [ ]:
offset

In [ ]:
# def Adam_calib(pix_no, pos):
#     a = -9.32768153e-15*pos+2.52898805e-09
#     b = 1.01644468e-10*pos +2.18827411e-05
#     c = 6.96885173e-07*pos-2.65767535e-01
#     d = -2.78778796e-03*pos+5.28655448e+02
#     return a*pix_no**3 + b*pix_no**2 + c*pix_no +d
    

In [ ]:

for s in range(8):
    x = 2048
    plt.plot(np.arange(x),dic['lq'][s,:], color=cs[s], alpha=0.2, lw=4, label = np.array(dic['Steps']).T[s][0])
    plt.plot(np.arange(x), Adam_calib(x, np.array(dic['Steps']).T[s][0]), ':', color=cs[s])
    plt.plot(np.arange(x), Andor_calib(x, np.array(dic['Steps']).T[s][0]), '--', color=cs[s])

plt.xlabel('Wavelength')
plt.ylabel('Pix Number')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Given Arrays (shaded), Calculated Function (:), Previous Calib (--)')

In [ ]:
np.array(dic['Steps']).T[s][0]